In this notebook, we explore active learning when already having a set of experimentally determined TCR-APL library pairs. The question is: which APLs should we measure for a new TCR, to be able to predict the remaining ones with highest performance.

In [1]:
%matplotlib inline

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from tqdm import tqdm
from itertools import product

In [3]:
import json
import os
import sys
sys.path.append('..')

from preprocessing import add_activation_thresholds, full_aa_features, get_aa_features 
from preprocessing import get_complete_dataset
                
from utils_al import train_classification_model, predict_classification_on_test, train_regression_model
from utils_al import predict_regression_on_test, evaluate_classification_models, evaluate_regression_models
from utils_al import get_aa_blosum, get_metrics_cls, get_metrics_reg

## Data Loading

In [4]:
#BASE_EPITOPE = 'VPSVWRSSL'
BASE_EPITOPE = 'SIINFEKL'

if BASE_EPITOPE == 'SIINFEKL':
    NORMALIZATION = 'AS'
    THRESHOLD = '46.9'
else:
    NORMALIZATION = 'pc'
    THRESHOLD = '66.09'

In [5]:
def load_data():
    data = get_complete_dataset()
    data = data[data['mut_pos']>=0]
    data = add_activation_thresholds(data)
    
    data['full_sample'] = data[['epitope', 'tcr']].apply(tuple, axis=1)
    data = data[data['normalization'] == NORMALIZATION]
    data = data[data['threshold'] == THRESHOLD]
    data = data[data['is_educated'] == True]
    aa_features = get_aa_features()
    features = full_aa_features(data, aa_features[['factors']], include_tcr=True)
    return data, features

In [6]:
def load_tumor_data():
    data = get_complete_dataset(BASE_EPITOPE)
    data = data[data['mut_pos']>=0]
    data = add_activation_thresholds(data, epitope=BASE_EPITOPE)
    data['full_sample'] = data[['epitope', 'tcr']].apply(tuple, axis=1)
    data = data[data['normalization'] == NORMALIZATION]
    data = data[data['threshold'] == THRESHOLD]
    aa_features = get_aa_features()
    features = full_aa_features(data, aa_features[['factors']], base_peptide=BASE_EPITOPE, include_tcr=True)
    return data, features

In [7]:
if BASE_EPITOPE == 'SIINFEKL':
    data, features = load_data()
else:
    data, features = load_tumor_data()

## Helper Functions

In [8]:
def train_classification_model(samples_train, seed):
    mask = data['full_sample'].isin(samples_train)
    data_train = data[mask]
    feat_train = features[mask]
    y_train = data_train['is_activated']
    clf = RandomForestClassifier(n_estimators=1000, random_state=seed).fit(feat_train, y_train)
    return clf

In [9]:
def predict_classification_on_test(clf, samples_train):
    mask = ~data['full_sample'].isin(samples_train)
    data_test = data[mask]
    feat_test = features[mask]
    
    y_test = data_test['is_activated']
    p_test = clf.predict_proba(feat_test)
    p_test = p_test[:, (1 if p_test.shape[1] == 2 else 0)]
    return p_test, y_test

In [10]:
def train_regression_model(samples_train, seed):
    mask = data['full_sample'].isin(samples_train)
    data_train = data[mask]
    feat_train = features[mask]
    y_train = data_train['activation']
    reg = RandomForestRegressor(n_estimators=250, max_features='sqrt', criterion='mae', random_state=seed).fit(feat_train, y_train)
    return reg

In [11]:
def predict_regression_on_test(reg, samples_train):
    mask = ~data['full_sample'].isin(samples_train)
    data_test = data[mask]
    feat_test = features[mask]
    
    y_test = data_test['activation']
    y_pred = reg.predict(feat_test)
    return y_pred, y_test

## Aquisition Functions

In [12]:
def add_random(samples_train, clf, reg, N):
    remaining_samples = data[~data['full_sample'].isin(samples_train)]
    remaining_samples = remaining_samples['full_sample']
    remaining_samples = remaining_samples.values
    new_samples = np.random.choice(remaining_samples, N, replace=False).tolist()
    return new_samples

In [13]:
def add_active(samples_train, clf, reg, N):
    p_test, _ = predict_classification_on_test(clf, samples_train)
    
    threshold = 0.5 # sum(p_test) / len(p_test)
    uncertainties = np.abs(p_test-threshold)
    idx_top_uncertain = uncertainties.argsort()[:N]
    data_test = data[~data['full_sample'].isin(samples_train)]
    new_samples = data_test.iloc[idx_top_uncertain]['full_sample'].tolist()
    return new_samples

In [14]:
def get_max_blosum_set():
    list_apls = []
    blosum_mat = get_aa_blosum()
    for idx, aa_base in enumerate(BASE_EPITOPE):
        new_aa_idx = blosum_mat.loc[aa_base].argmin()

        new_aa = blosum_mat.columns[new_aa_idx]
        new_epitope = BASE_EPITOPE[:idx] + new_aa + BASE_EPITOPE[idx+1:]
        list_apls.append(new_epitope)
    return list_apls

## Training Loops

In [15]:
def run_learning_loop(tcr, method_data_aquisition, 
                      N=8, M=5, seed=0):
    samples_train = data[data['tcr']!=tcr]['full_sample'].values.tolist()
    
    results = {}
    for metric_name in list(metrics_class[0].keys()) + list(metrics_class[1].keys()) + list(metrics_reg.keys()):
        results[metric_name] = []
    
    clf = train_classification_model(samples_train, seed)
    reg = train_regression_model(samples_train, seed)
    
    samples_tcr = []
    for idx in range(M):
        new_samples = method_data_aquisition(samples_train, clf, reg, N)
        samples_train += new_samples
        samples_tcr += [el[0] for el in new_samples]
        
        clf = train_classification_model(samples_train, seed)
        p_test, c_truth = predict_classification_on_test(clf, samples_train)
        evaluate_classification_models(c_truth, p_test, metrics_class, results, idx)
        
        reg = train_regression_model(samples_train, seed)
        y_test, y_truth = predict_regression_on_test(reg, samples_train)
        evaluate_regression_models(y_truth, y_test, metrics_reg, results, idx)
    
    path_apls = f'../results/active_learning/across/{BASE_EPITOPE}_selectedAPLs_'
    path_apls += f'{method_data_aquisition.__name__}.csv'
    if not os.path.exists(path_apls):
        with open(path_apls, 'w') as f:
            f.write('seed,tcr,apls\n')

    with open(path_apls, 'a') as f:
        f.write(f'{seed},{tcr},{str(samples_tcr)}\n')
    return results

In [16]:
def run_experiment(method_data_aquisition, N, M, do_reload=False):
    global data
    global features
    results = {}
    for metric_name in list(metrics_reg.keys()) + list(metrics_class[0].keys()) + list(metrics_class[1].keys()):
        results[metric_name] = []

    for i in tqdm(range(10)):
        for tcr in data['tcr'].unique():
            scores = run_learning_loop(tcr, method_data_aquisition, N=N, M=M, seed=i)
            for name, score in scores.items():
                results[name] += [[tcr] + val for val in score]
            if do_reload:
                data, features = load_data()
    return results

## Plotting Functions

In [17]:
def do_plots(summary, palette, x_ticks, x_labels, plot_type):
    sns.set(font_scale=2)
    sns.set_style('white')

    for name in list(list(metrics_class[0].keys()) + list(metrics_class[1].keys()) + list(metrics_reg.keys())):
        plt.figure(figsize=(12, 8))
        dfs_results = []
        for method in summary.keys():
            df = pd.DataFrame(summary[method][name])
            df.columns = ['tcr', 'iteration', name]
            df['method'] = method
            dfs_results.append(df)
        df_joint = pd.concat(dfs_results)
        plot = plot_type(data=df_joint, x='iteration', y=name, hue='method', palette=palette)
              
        
        sns.despine(bottom=False, left=False)

        plot.set(xlabel='Amount Samples')
        if len(name)<=3:
            plot.set(ylabel=name.upper())

        plt.legend(title='Sampling Method')
        plot.set_xticks(x_ticks)
        plot.set_xticklabels(x_labels)
        plot.figure.savefig(f'../figures/active_learning/active_across_class_{name}_N-{N}.png', dpi=300)
        plot.figure.savefig(f'../figures/active_learning/active_across_class_{name}_N-{N}.pdf', dpi=300)
        plt.show()

## Evaluation

In [18]:
def save_run_experiment(name, aquisition_method, do_reload=False):
    path_out = f'../results/active_learning/across/{BASE_EPITOPE}_crossTCR_FULL_{name}_{N}_{M}.json'
    if not os.path.exists(path_out) or REDO:
        results_exp = run_experiment(aquisition_method, N, M, do_reload)
        with open(path_out, 'w') as json_file:
            json.dump(results_exp, json_file)
    else:
        with open(path_out) as f:
            results_exp = json.load(f)
    return results_exp

In [19]:
metrics_reg = get_metrics_reg()
metrics_class = get_metrics_cls()

In [20]:
N = 8
M = 10
REDO = False

In [ ]:
results_random = save_run_experiment('random', add_random)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

In [ ]:
results_active = save_run_experiment('active', add_active)

In [ ]:
def read_greedy_bound():
    path_in = f'../results/active_learning/across/greedy/{BASE_EPITOPE}/'
    res_files = [path_in + f for f in os.listdir(path_in) if os.path.isfile(os.path.join(path_in, f))]

    results_upper = {}
    for name in list(metrics_reg.keys()) + list(metrics_class[0].keys()) + list(metrics_class[1].keys()):
        results_upper[name] = []

    for path_file in res_files:
        with open(path_file) as f:
            res_tmp = json.load(f)
        for mtc, vals in res_tmp.items():
            results_upper[mtc] += vals
    return results_upper

In [ ]:
summary_full = {
    'active': results_active,
    'random': results_random,
    #'upper': read_greedy_bound()
}

In [ ]:
palette = {'active': 'tab:red', 'upper': 'tab:olive', 'random': 'tab:cyan'}
x_ticks = list(range(0, M))
x_labels = ['9'] + [str(9+(i+1)*N) for i in range(M-1)]

do_plots(summary_full, palette, x_ticks, x_labels, sns.lineplot)